In [10]:
import numpy as np
from scipy.io import loadmat

# Load the data
data = loadmat('Iosipescu_isotropic.mat')
X1 = data['X1']
X2 = data['X2']
L = data['L'].item()
w = data['w'].item()
t = data['t'].item()
F = data['F'].item()
Eps1 = data['Eps1']
Eps2 = data['Eps2']
Eps6 = data['Eps6']

# Units in N and mm → MPa for stiffness
# Direct identification of shear modulus
G = F / (w * t * np.mean(Eps6))

# I: First set of virtual fields
A = np.zeros((2, 2))
B = np.zeros(2)

# First virtual field
A[0, 0] = np.mean(Eps6)
A[0, 1] = -np.mean(Eps6)
B[0] = 2 * F / (w * t)

# Second virtual field
A[1, 0] = -np.mean(Eps1 * (L - 2 * X1) * X2)
A[1, 1] = -np.mean(Eps2 * (L - 2 * X1) * X2)
B[1] = F * L**2 / (6 * w * t)

Q = np.linalg.inv(A) @ B
Nu = [Q[1] / Q[0]]
E = [Q[0] * (1 - Nu[0]**2)]

# II: Second set of virtual fields
AA = np.zeros((2, 2))
BB = np.zeros(2)

# First virtual field
AA[0, 0] = np.mean(Eps6)
AA[0, 1] = -np.mean(Eps6)
BB[0] = 2 * F / (w * t)

# Second virtual field
AA[1, 0] = -np.mean(Eps1 * (L - 2 * X1) * X2) - 0.5 * np.mean(Eps6 * X1 * (L - X1))
AA[1, 1] = -np.mean(Eps2 * (L - 2 * X1) * X2) + 0.5 * np.mean(Eps6 * X1 * (L - X1))
BB[1] = 0

QQ = np.linalg.inv(AA) @ BB
Nu.append(QQ[1] / QQ[0])
E.append(QQ[0] * (1 - Nu[1]**2))

# III: Third set of virtual fields
AAA = np.zeros((2, 2))
BBB = np.zeros(2)

# First virtual field
AAA[0, 0] = np.mean(Eps6)
AAA[0, 1] = -np.mean(Eps6)
BBB[0] = 2 * F / (w * t)

# Second virtual field
AAA[1, 0] = -np.mean(Eps2 * X1 * (L - X1)) - 0.5 * np.mean(Eps6 * (L - 2 * X1) * X2)
AAA[1, 1] = -np.mean(Eps1 * X1 * (L - X1)) + 0.5 * np.mean(Eps6 * (L - 2 * X1) * X2)
BBB[1] = 0

QQQ = np.linalg.inv(AAA) @ BBB
Nu.append(QQQ[1] / QQQ[0])
E.append(QQQ[0] * (1 - Nu[2]**2))

# Output

print("Shear modulus G (MPa):", "{:.4e}".format(G))
print("Identified Young's moduli (MPa):", " ; ".join("{:.3e}".format(e) for e in E))
print("Identified Poisson's ratios:", " ; ".join("{:.3e}".format(nu) for nu in Nu))


Shear modulus G (MPa): 1.9999e+04
Identified Young's moduli (MPa): 5.201e+04 ; 5.202e+04 ; 5.186e+04
Identified Poisson's ratios: 3.004e-01 ; 3.005e-01 ; 2.965e-01


In [45]:
from scipy.interpolate import RegularGridInterpolator
import os

# =============================================================================
# 4. Load FEM Data and Build Interpolation Functions
# =============================================================================
dir_path = ".."
FEM_dataset = "100x100mm.dat"
L_max = 100.0

fem_file = os.path.join(dir_path, r"data_fem", FEM_dataset)
data = np.loadtxt(fem_file)
X_val      = data[:, :2]
u_val      = data[:, 2:4]
strain_val = data[:, 4:7]
stress_val = data[:, 7:10]
solution_val = np.hstack((u_val, stress_val))

n_mesh_points = int(np.sqrt(X_val.shape[0]))
x_grid = np.linspace(0, L_max, n_mesh_points)
y_grid = np.linspace(0, L_max, n_mesh_points)

def create_interpolation_fn(data_array):
    num_components = data_array.shape[1]
    interpolators = []
    for i in range(num_components):
        interp = RegularGridInterpolator(
            (x_grid, y_grid),
            data_array[:, i].reshape(n_mesh_points, n_mesh_points).T,
        )
        interpolators.append(interp)
    def interpolation_fn(x):
        return np.array([interp((x[:, 0], x[:, 1])) for interp in interpolators]).T
    return interpolation_fn

solution_fn = create_interpolation_fn(solution_val)
strain_fn   = create_interpolation_fn(strain_val)

In [85]:
import numpy as np
import pandas as pd
import os

dir_path = ".."

# --- 1) Adding Gaussian noise to the data ---
noise_level = 1e-4
Eps1 += np.random.normal(0, noise_level, Eps1.shape)
Eps2 += np.random.normal(0, noise_level, Eps2.shape)
Eps6 += np.random.normal(0, noise_level, Eps6.shape)

# --- 2) Problem parameters from your FEniCS code ---
L = 100.0   # mm
m = 0.3     # N/mm (side‐loading slope)
b = 50.0    # N (side‐loading intercept)

# --- 3) Compute total shear load on the right boundary ---
#    ∫[0→L] (m y + b) dy = m L^2/2 + b L
F = m*L**2/2 + b*L

def calc_parameters(Eps1, Eps2, Eps6):
        """
        Calculate the E and ν parameters from the strain data.
        """
        # build A and B
        A = np.zeros((2, 2))
        B = np.zeros(2)

        # Virtual field 1:  u1*=0, u2*=-x2  ⇒ ε1*=0, ε2*=-1
        A[0, 0] = np.mean(Eps2)*L**2     
        A[0, 1] = np.mean(Eps1)*L**2      
        B[0]    = 0        

        # Virtual field 2:  u1*=x1, u2*=0  ⇒ ε1*=1, ε2*=0
        A[1, 0] = np.mean(Eps1)*L**2       # ∑ ε₁ sᵢ
        A[1, 1] = np.mean(Eps2)*L**2      # ∑ ε₂ sᵢ
        B[1]    = F * L                # ∑ σ₁ lᵢ

        # solve for Q = [Q11, Q12]
        Q11, Q12 = np.linalg.solve(A, B)

        # retrieve E and ν
        μ_id = (Q11 - Q12)/2
        λ_id = Q12
        Nu  = λ_id / (2*(μ_id + λ_id))
        E  = μ_id*(3*λ_id + 2*μ_id)/(λ_id + μ_id)
        return E, Nu

def load_data(DIC_dataset_path, DIC_dataset_number, L_max, strain_fn, num_measurments, noise_magnitude, DIC_region=[0,0,L_max,L_max]):
    """
    Load the DIC data from the specified path.
    """
    if DIC_dataset_path != "no_dataset":
        dic_path = os.path.join(dir_path, DIC_dataset_path)
        dic_number = DIC_dataset_number
        X_dic = pd.read_csv(os.path.join(dic_path, "x", f"x_{dic_number}.csv"), delimiter=";").dropna(axis=1).to_numpy()
        Y_dic = pd.read_csv(os.path.join(dic_path, "y", f"y_{dic_number}.csv"), delimiter=";").dropna(axis=1).to_numpy()
        Ux_dic = pd.read_csv(os.path.join(dic_path, "ux", f"ux_{dic_number}.csv"), delimiter=";").dropna(axis=1).to_numpy().T.reshape(-1, 1)
        Uy_dic = pd.read_csv(os.path.join(dic_path, "uy", f"uy_{dic_number}.csv"), delimiter=";").dropna(axis=1).to_numpy().T.reshape(-1, 1)
        E_xx_dic = pd.read_csv(os.path.join(dic_path, "exx", f"exx_{dic_number}.csv"), delimiter=";").dropna(axis=1).to_numpy().T.reshape(-1, 1)
        E_yy_dic = pd.read_csv(os.path.join(dic_path, "eyy", f"eyy_{dic_number}.csv"), delimiter=";").dropna(axis=1).to_numpy().T.reshape(-1, 1)
        E_xy_dic = pd.read_csv(os.path.join(dic_path, "exy", f"exy_{dic_number}.csv"), delimiter=";").dropna(axis=1).to_numpy().T.reshape(-1, 1)
        
        region_mask = np.logical_and(
            np.logical_and(X_dic >= DIC_region[0], X_dic <= DIC_region[2]),
            np.logical_and(Y_dic >= DIC_region[1], Y_dic <= DIC_region[3])
        )
        X_dic = X_dic[region_mask]
        Y_dic = Y_dic[region_mask]
        E_xx_dic = E_xx_dic[region_mask.T.reshape(-1,1)]
        E_yy_dic = E_yy_dic[region_mask.T.reshape(-1,1)]
        E_xy_dic = E_xy_dic[region_mask.T.reshape(-1,1)]
        X_DIC_input = np.hstack([X_dic, Y_dic])
        DIC_data = np.stack([E_xx_dic, E_yy_dic, E_xy_dic], axis=-1)
    else:
        X_DIC_input = [np.linspace(DIC_region[0], DIC_region[2], num_measurments),
                       np.linspace(DIC_region[1], DIC_region[3], num_measurments)]
        X_DIC_input = np.meshgrid(*X_DIC_input)
        X_DIC_input = np.stack([x.ravel() for x in X_DIC_input], axis=-1) 
        DIC_data = strain_fn(X_DIC_input)
        DIC_data += np.random.normal(0, noise_magnitude, DIC_data.shape)
    return X_DIC_input, DIC_data




In [93]:

prop = 0
DIC_region = [prop*L, prop*L, (1-prop)*L, (1-prop)*L]
X_DIC_input, DIC_data = load_data("no_dataset", 0, L, strain_fn, 10, 1e-6, DIC_region=DIC_region)
Eps1 = DIC_data[:, 0].reshape(-1, 1)
Eps2 = DIC_data[:, 1].reshape(-1, 1)
Eps6 = DIC_data[:, 2].reshape(-1, 1)

E, Nu = calc_parameters(Eps1, Eps2, Eps6)
print(f"Identified:  E = {E/1e3:.4f} GPa,   ν = {Nu:.4f}")

Identified:  E = 209.1031 GPa,   ν = 0.3015


In [ ]:
camera_resolution = "2MP"
DIC_dataset_path = f"2_noise_study/data_dic/{camera_resolution}/1noise"
prop = 0
DIC_region = [prop*L, prop*L, (1-prop)*L, (1-prop)*L]

E_list = []
Nu_list = []
for dic_number in range(1, 11):
    X_DIC_input, DIC_data = load_data(DIC_dataset_path, dic_number, L, strain_fn, 4, 1e-6, DIC_region=DIC_region)
    Eps1 = DIC_data[:, 0].reshape(-1, 1)
    Eps2 = DIC_data[:, 1].reshape(-1, 1)
    Eps6 = DIC_data[:, 2].reshape(-1, 1)

    E, Nu = calc_parameters(Eps1, Eps2, Eps6)
    E_list.append(E)
    Nu_list.append(Nu)

E_list = np.array(E_list)
Nu_list = np.array(Nu_list)

print(f"Identified (mean +/- std):  E = {np.mean(E_list)/1e3:.4f} ± {np.std(E_list)/1e3:.4f} GPa,   ν = {np.mean(Nu_list):.4f} ± {np.std(Nu_list):.4f}")

Identified (mean +/- std):  E = 210.2148 ± 0.0885 GPa,   ν = 0.2989 ± 0.0002


In [99]:
import json
import numpy as np

# Assuming load_data and calc_parameters are defined elsewhere
# Define your variables and functions here

resolutions = ["5MP", "2MP", "0.4MP"]
prop = 0
precision = 4 

# Initialize lists to store VFM results for each resolution
E_VFM_means = []
E_VFM_stds = []
Nu_VFM_means = []
Nu_VFM_stds = []

for camera_resolution in resolutions:
    DIC_dataset_path = f"2_noise_study/data_dic/{camera_resolution}/1noise"
    DIC_region = [prop*L, prop*L, (1-prop)*L, (1-prop)*L]

    E_list = []
    Nu_list = []
    for dic_number in range(1, 11):
        X_DIC_input, DIC_data = load_data(DIC_dataset_path, dic_number, L, strain_fn, 4, 1e-6, DIC_region=DIC_region)
        Eps1 = DIC_data[:, 0].reshape(-1, 1)
        Eps2 = DIC_data[:, 1].reshape(-1, 1)
        Eps6 = DIC_data[:, 2].reshape(-1, 1)

        E, Nu = calc_parameters(Eps1, Eps2, Eps6)
        E_list.append(E)
        Nu_list.append(Nu)

    E_list = np.array(E_list)
    Nu_list = np.array(Nu_list)

    # Calculate mean and std for E and Nu
    E_mean = np.mean(E_list) / 1e3
    E_std = np.std(E_list) / 1e3
    Nu_mean = np.mean(Nu_list)
    Nu_std = np.std(Nu_list)

    # Append the results to the lists
    E_VFM_means.append(round(E_mean, precision))
    E_VFM_stds.append(round(E_std, precision))
    Nu_VFM_means.append(round(Nu_mean, precision))
    Nu_VFM_stds.append(round(Nu_std, precision))

# Load the existing JSON file
with open("../2_noise_study/results/summary.json", 'r') as json_file:
    json_data = json.load(json_file)

# Append the VFM field to the JSON structure for each resolution
json_data["E"]["VFM"] = {"mean": E_VFM_means, "std": E_VFM_stds}
json_data["nu"]["VFM"] = {"mean": Nu_VFM_means, "std": Nu_VFM_stds}

# Save the updated JSON structure back to the file
with open("../2_noise_study/results/summary.json", 'w') as json_file:
    json.dump(json_data, json_file, indent=4)
